In [1]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
import os
import imutils
from tqdm import tqdm
from copy import deepcopy
import math
import time
from multiprocessing import Pool
import pandas as pd

In [2]:
from util_db import search_deck, search_card
from match_scale import match_scale, match_scale_mp
from match_card import match_card, match_card_v
from util_trim import get_card_name, get_paste, get_tidy
from util_tools import dist, sort_pt
from util_trim import trans_raw, trans_dup, trans_group, trans_sort, trans_name, trans_paste

In [3]:
top_card = [
23434538,
14558127,
8267140,
15693423,
24224830,
54693926,
10045474,
94145021,
41420027,
18144507,
27204311,
90448279,
84211599
]

top_name = [
'증식의 G O',   #O
'하루 우라라 X', # X
'코즈믹 싸이크론 X', # X
'길항승부 X', # X
'무덤의 지명자 O', # O
'명왕결계파 X', # X
'무한포영 X ', # X
'드롤 & 로크 버드 X', # X
'신의 심판 X', # X
'해피의 깃털 O', # O
'원시생명체 니비루 X', # X
'네가로기어 아제우스 O', # O
'졸부와 겸허의 항아리 X' # X
]

In [9]:
dir_card_img = 'C:/ML_YGO/YGO_DB/'
dir_card_list = []
for digit_card in top_card :
    dir_card = f'{dir_card_img}{digit_card}.jpg'
    dir_card_list.append(dir_card)
#######################################################
card_sample = 1
dir_card = dir_card_list[card_sample]
card_name = top_name[card_sample]
print(f'Name of Card : {card_name}')
#######################################################
deck_sample = 0
dir_deck = search_deck(deck_sample)
deck_code = os.path.splitext(os.path.basename(dir_deck))[0]
print(f'Name of Deck : {deck_code}')
#######################################################
deck_width = 400
card_width = match_scale(dir_card, dir_deck, deck_width)
print(f'Width of Deck : {deck_width}')
print(f'Width of Card : {card_width}')
#######################################################
threshold = 0.8
dup_tol = card_width/3
sort_round = deck_width/20
#######################################################

Name of Card : 하루 우라라 X
Name of Deck : 1004-001
Ideal card image size is : 21 x 32
Width of Deck : 400
Width of Card : 32


In [20]:
img_deck = cv2.imread(dir_deck)
img_deck = imutils.resize(img_deck, width = deck_width)
img_gray = cv2.cvtColor(img_deck, cv2.COLOR_BGR2GRAY)

img_card = cv2.imread(dir_card, 0)

best_match = None
sc1 = int((deck_width / 10) * 0.8 )
sc2 = int((deck_width / 10) * 1.2 )
sc_dif = int(abs(sc2-sc1)/1)
for scale in np.linspace(sc1, sc2, sc_dif):
    resized_img_card = imutils.resize(img_card, width = int(scale))    
    res = cv2.matchTemplate(img_gray, resized_img_card, cv2.TM_SQDIFF) # TM_CCOEFF_NORMED TM_SQDIFF
    min_val, _, min_loc, _ = cv2.minMaxLoc(res)
    if best_match is None or min_val <= best_match[0]:
        ideal_scale=scale 
        best_match = [min_val, min_loc, ideal_scale]
ideal_w = int(img_card.shape[1]*ideal_scale / img_card.shape[0])
ideal_h = int(ideal_scale)
print(f"Ideal card image size is : {ideal_w} x {ideal_h}")
print(f'Width of Card : {card_width}')
card_width = ideal_h

# img_card = cv2.imread(dir_card, 0)
img_card = imutils.resize(img_card, width = card_width)
res = cv2.matchTemplate(img_gray, img_card, cv2.TM_CCOEFF_NORMED)
loc=np.where(res >= threshold)

lengths = [len(elem) for elem in loc]
if all(length == 0 for length in lengths):
    actual_length = 0
else:
    actual_length = max(lengths)

if actual_length >0 :
    real_width = card_width 
else :
    real_width = None

print(actual_length, real_width)

Ideal card image size is : 27 x 39
Width of Card : 39
0 None


In [21]:
card_width_list = []
for i in range(len(dir_card_list)):
    dir_card = dir_card_list[i]
    # card_name = top_name[i]
    card_width = match_scale_mp(dir_card, dir_deck, deck_width, threshold)
    # print(f'Card Width for {card_name} : {card_width}')
    if card_width == None:
        pass
    else:
        card_width_list.append(card_width)
average = sum(card_width_list) / len(card_width_list)
print(card_width_list, average)

[39, 39, 39] 39.0


In [12]:
count = 0
total = 0
for item in card_width_list:
    if item is not None:
        total += item
        count += 1

if count > 0:
    average = total / count
    print(f"The average of the list is: {average}")
else:
    print("The list contains no valid elements to average.")

The average of the list is: 39.0
